### Notebook to produce ncdf files of era5 data associated to the different crops of wet, dry and intrusions of wet into dry and dry into wet
#### author: Claudia Acquistapace
#### date: 03/02/2023


##### explanation of the nomenclature of the files
wet = blue
dry = grey
wet crops = "only blue"
dry crops = 'only grey"
intrusion of wet crops into dry crops = "blue in grey"
intrusion of dry crops into wet crops = "grey in blue"


In [16]:
a = [1,2,3,4]
print(len(a))

4


In [21]:
import xarray as xr
import numpy as np
import pandas as pd
from pathlib import Path
import glob
import matplotlib.pyplot as plt
from matplotlib import rcParams
import matplotlib
from datetime import datetime, timedelta
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from matplotlib import rcParams
from warnings import warn
import datetime as dt
from scipy import interpolate
import matplotlib as mpl
import os.path
import itertools
import os.path
from random import randrange
import matplotlib.patches as mpatches

# definition of the areas to be analyzed for DC's paper
string_areas = ['wet_only', 'wet_in_dry', 'dry_in_wet'] # 'dry', 

# define index that is needed to read image id string from image file name
ind_start = len('/p/scratch/deepacf/kiste/DC/dataset/barbados/barbados_leif/1/')

# path and filename of ERA5 data
path_era5 = '/data/mod/era/era5/tropical_atlantic'

# defining path out where to store the data
path_out = '/net/ostro/ML_work_DC/era5_dry_wet/'

# reading one era5 data to extract height dimension (needed for defining output matrices of era5 data)
profile_era5_start = xr.open_dataset(path_era5+'/2017/03/profb_presslev_20170322T1300.nc')
height = profile_era5_start.level.values


# loop on the areas 
for ind_area, area in enumerate(string_areas):
    
    print(area)
    
    # setting input path and filename
    if area == 'wet_only':
        info_file = pd.read_csv('/data/sat/goes-r-abi/ATLANTIC/dry_wet_exp/only_blue.csv')
        #path_lat_lon_files = 'data/sat/goes-r-abi/ATLANTIC/dry_wet_exp/lat_lon_onlyblue/'
        path_lat_lon_files = 'data/sat/goes-r-abi/ATLANTIC/dry_wet_exp/lat_lon_onlyblue/'
    elif area == 'dry_in_wet':
        info_file = pd.read_csv('/data/sat/goes-r-abi/ATLANTIC/dry_wet_exp/gray_in_blue.csv')
        path_lat_lon_files = 'data/sat/goes-r-abi/ATLANTIC/dry_wet_exp/lat_lon_gray_in_blue/'
    elif area == 'wet_in_dry':
        info_file = pd.read_csv('/net/ostro/ML_work_DC/intrusion1_embedding_space.csv')
        path_lat_lon_files = '/net/ostro/ML_work_DC/lat_lon_dry_wet/'
    
    wet_dry_info = info_file.to_xarray()
    
    # reading the number of images in the selected file 
    n_images = len(wet_dry_info.index.values)

    # initializing output variables
    SST = np.zeros(n_images)
    im_type = np.zeros(n_images)
    im_name_arr = np.zeros(n_images)
    TCWV =np.zeros(n_images)
    TCC = np.zeros(n_images)
    TCLW =np.zeros(n_images)
    TCRW = np.zeros(n_images)
    CBH = np.zeros(n_images)
    SST_std = np.zeros(n_images)
    TCWV_std = np.zeros(n_images)
    TCC_std = np.zeros(n_images)
    TCLW_std =np.zeros(n_images)
    TCRW_std = np.zeros(n_images)
    CBH_std = np.zeros(n_images)
    q_profile = np.zeros((n_images, len(height)))
    t_profile = np.zeros((n_images, len(height)))
    q_profile_std = np.zeros((n_images, len(height)))
    t_profile_std = np.zeros((n_images, len(height)))

    w_profile = np.zeros((n_images, len(height)))
    rh_profile = np.zeros((n_images, len(height)))
    w_profile_std = np.zeros((n_images, len(height)))
    rh_profile_std = np.zeros((n_images, len(height)))
    clwc_profile = np.zeros((n_images, len(height)))
    clwc_profile_std = np.zeros((n_images, len(height)))
    cc_profile = np.zeros((n_images, len(height)))
    cc_profile_std = np.zeros((n_images, len(height)))
    u_profile = np.zeros((n_images, len(height)))
    u_profile_std = np.zeros((n_images, len(height)))
    v_profile = np.zeros((n_images, len(height)))
    v_profile_std = np.zeros((n_images, len(height)))
    SST.fill(np.nan)
    TCWV.fill(np.nan)
    TCC.fill(np.nan)
    TCLW.fill(np.nan)
    TCRW.fill(np.nan)
    CBH.fill(np.nan)
    SST_std.fill(np.nan)
    TCWV_std.fill(np.nan)
    TCC_std.fill(np.nan)
    TCLW_std.fill(np.nan)
    TCRW_std.fill(np.nan)
    CBH_std.fill(np.nan)
    q_profile.fill(np.nan)
    t_profile.fill(np.nan)
    q_profile_std.fill(np.nan)
    t_profile_std.fill(np.nan)
    w_profile.fill(np.nan)
    rh_profile.fill(np.nan)
    w_profile_std.fill(np.nan)
    rh_profile_std.fill(np.nan)
    clwc_profile.fill(np.nan)
    clwc_profile_std.fill(np.nan)
    u_profile.fill(np.nan)
    u_profile_std.fill(np.nan)
    v_profile.fill(np.nan)
    v_profile_std.fill(np.nan)
    im_name_arr.fill(np.nan)

    # additional lists of info for the images
    im_name_arr = []
    im_type = np.zeros(n_images)
    id_lat_lon_arr = []
    datetime_arr = []
    
    
    # loop on the images to collect era 5 data if they are present
    for ind_images in range(n_images):

        # read image id string and id for lat/lon
        id_image = wet_dry_info.location.values[ind_images][ind_start:ind_start+14]
        im_name_arr.append(id_image)
        
        id_lat_lon = wet_dry_info.location.values[ind_images][ind_start+14:ind_start+16]
        id_lat_lon_arr.append(id_lat_lon)
        
        print(ind_images, id_lat_lon, im_name)

        # reading image type (wet or dry)
        type_im = wet_dry_info.dry_wet.values[ind_images]
        if type_im == 'dry':
            im_type[ind_images] = 1.
        else:
            im_type[ind_images] = 0.

        # reading year, month, day to build name of the image and find corresponding era5 data
        datetime_value = pd.to_datetime(wet_dry_info.datetime.values[ind_images])
        datetime_arr.append(datetime_value)
        
        yy = str(datetime_value.year)
        mm = str(datetime_value.month)
        dd = str(datetime_value.day)
        hh = str(datetime_value.hour)
        mn = str(datetime_value.minute)

        # reading lat/lon
        # building era5 path
        if len(dd) == 1:
            dd = '0'+dd
        if len(hh) == 1:
            hh = '0'+hh
        if len(mm) == 1:
            mm = '0'+mm
        if len(mn) == 1:
            mn = '0'+mn

        # constructing date of the selected image (needed for era5)
        date = yy+mm+dd+hh+mn+'00'
        
        # assigning path for era5 based on the date
        era5_day_path = path_era5+'/'+yy+'/'+mm+'/'
        
        # proceed only if there are era5 data for the selected date
        if os.path.isfile(era5_day_path+'profb_presslev_'+yy+mm+dd+'T'+hh+'00.nc') * \
         os.path.isfile(era5_day_path+'surfskinvarb_'+yy+mm+dd+'T'+hh+'00.nc'):

            # find how many lat/lons we have associated to the id-image
            if id_lat_lon != '_h':
                file_lat =  '/net/ostro/ML_work_DC/lat_lon_onlyblue/'+id_image+'_lat'+id_lat_lon+'.npy'
                file_lon =  '/net/ostro/ML_work_DC/lat_lon_onlyblue/'+id_image+'_lon'+id_lat_lon+'.npy'
            else:
                file_lat =  '/net/ostro/ML_work_DC/halo_lat_lon/halo_lat.npy'
                file_lon =  '/net/ostro/ML_work_DC/halo_lat_lon/halo_lon.npy'              
            #print(file_lat, file_lon)
            
            # if lat and lon files are present then process the data
            if (os.path.isfile(file_lat) * os.path.isfile(file_lon)):

                # reading lats/lons for the id_image
                lat_data = np.load(file_lat)
                lon_data = np.load(file_lon)
                lat_max = np.nanmax(lat_data)
                lat_min = np.nanmin(lat_data)
                lon_min = np.nanmin(lon_data)
                lon_max = np.nanmax(lon_data)

                # reading era
                profile_era5 = xr.open_dataset(era5_day_path+'profb_presslev_'+yy+mm+dd+'T'+hh+'00.nc')
                surface_era5 = xr.open_dataset(era5_day_path+'surfskinvarb_'+yy+mm+dd+'T'+hh+'00.nc')

                # selecting the area corresponding to the crop
                surface_crop = surface_era5.where((surface_era5.latitude > lat_min)*(surface_era5.latitude <= lat_max) \
                                                * (surface_era5.longitude > lon_min) *(surface_era5.longitude <= lon_max))

                profiles_crop = profile_era5.where((profile_era5.latitude > lat_min)*(profile_era5.latitude <= lat_max) \
                                                * (profile_era5.longitude > lon_min)*(profile_era5.longitude <= lon_max))

                # making a plot of the era5 domain and the selected box
                #fig, axs = plt.subplots(figsize=(10,10), constrained_layout=True)
                #surface_era5.tcwv.plot(x='longitude', y='latitude')
                #rect=mpatches.Rectangle((lon_max, lat_min),lon_min-lon_max,lat_max-lat_min,
                #                fill = False,
            #                    color = "purple",
            #                    linewidth = 2)
            #    plt.gca().add_patch(rect)
            #    fig.savefig(path_out+id_image+'_'+type_im+'.png')

                # calculating mean profiles and mean surface variables in the selected domain
                SST[ind_images] = surface_crop.sst.mean(skipna='True')
                SST_std[ind_images] = surface_crop.sst.std(skipna='True')

                TCWV[ind_images] = surface_crop.tcwv.mean(skipna='True')
                TCWV_std[ind_images] = surface_crop.tcwv.std(skipna='True')

                TCC[ind_images] = surface_crop.tcc.mean(skipna='True')
                TCC_std[ind_images] = surface_crop.tcc.std(skipna='True')

                TCLW[ind_images] = surface_crop.tclw.mean(skipna='True')
                TCLW_std[ind_images] = surface_crop.tclw.std(skipna='True')

                TCRW[ind_images] = surface_crop.tcrw.mean(skipna='True')
                TCRW_std[ind_images] = surface_crop.tcrw.std(skipna='True')

                CBH[ind_images] = surface_crop.cbh.mean(skipna='True')
                CBH_std[ind_images] = surface_crop.cbh.std(skipna='True')

                q_profile[ind_images,:] = profiles_crop.q.mean(dim=('longitude', 'latitude'), skipna='True')
                q_profile_std[ind_images,:] = profiles_crop.q.std(dim=('longitude', 'latitude'), skipna='True')
                t_profile[ind_images,:] = profiles_crop.t.mean(dim=('longitude', 'latitude'), skipna='True')
                t_profile_std[ind_images,:] = profiles_crop.t.std(dim=('longitude', 'latitude'), skipna='True')
                w_profile[ind_images,:] = profiles_crop.w.mean(dim=('longitude', 'latitude'), skipna='True')
                w_profile_std[ind_images,:] = profiles_crop.w.std(dim=('longitude', 'latitude'), skipna='True')
                u_profile[ind_images,:] = profiles_crop.u.mean(dim=('longitude', 'latitude'), skipna='True')
                u_profile_std[ind_images,:] = profiles_crop.u.std(dim=('longitude', 'latitude'), skipna='True')
                v_profile[ind_images,:] = profiles_crop.v.mean(dim=('longitude', 'latitude'), skipna='True')
                v_profile_std[ind_images,:] = profiles_crop.v.std(dim=('longitude', 'latitude'), skipna='True')
                rh_profile[ind_images,:] = profiles_crop.r.mean(dim=('longitude', 'latitude'), skipna='True')
                rh_profile_std[ind_images,:] = profiles_crop.r.std(dim=('longitude', 'latitude'), skipna='True')
                clwc_profile[ind_images,:] = profiles_crop.clwc.mean(dim=('longitude', 'latitude'), skipna='True')
                clwc_profile_std[ind_images,:] = profiles_crop.clwc.std(dim=('longitude', 'latitude'), skipna='True')
                cc_profile[ind_images,:] = profiles_crop.cc.mean(dim=('longitude', 'latitude'), skipna='True')
                cc_profile_std[ind_images,:] = profiles_crop.cc.std(dim=('longitude', 'latitude'), skipna='True')

    # storing data in a ncdf file for each area
    crop_data = xr.Dataset(
    data_vars={
        'im_names': (('n_crops',), im_name_arr, {'long_name': 'Names of image', 'units':''}),
        "SST": (('n_crops',), SST, {'long_name': 'Sea surface temperature ', 'units':'K', "standard_name": "SST"}),
        'SST_std':(('n_crops',), SST_std, {'long_name': 'Sea suface temperature standard deviation', 'units':'K', "standard_name": "SST_standard_dev"}),
        "TCWV": (('n_crops',), TCWV, {'long_name': 'Total column water vapor ', 'units':'kg m**-2', "standard_name": "TCWV"}),
        'TCWV_std':(('n_crops',), TCWV_std, {'long_name': 'Total column water vapor standard deviation', 'units':'kg m**-2', "standard_name": "TCWV_standard_dev"}),
        "TCC": (('n_crops',), TCC, {'long_name': 'Total cloud cover', 'units':'', "standard_name": "TCC"}),
        'TCC_std':(('n_crops',), TCC_std, {'long_name': 'Total cloud cover standard deviation', 'units':'', "standard_name": "TCC_standard_dev"}),
        "TCLW": (('n_crops',), TCLW, {'long_name': 'Total column cloud liquid water', 'units':'Kg m**-2', "standard_name": "TCLW"}),
        'TCLW_std':(('n_crops',), TCLW_std, {'long_name': 'Total column cloud liquid water standard deviation', 'units':'Kg m**-2', "standard_name": "TCLW_standard_dev"}),
        "TCRW": (('n_crops',), TCRW, {'long_name': 'Total column rain water', 'units':'Kg m**-2', "standard_name": "TCRW"}),
        'TCRW_std':(('n_crops',), TCRW_std, {'long_name': 'Total column rain water standard deviation', 'units':'Kg m**-2', "standard_name": "TCRW_standard_dev"}),
        'im_type':(('n_crops',), im_type, {'long_name': 'type of image (wet or dry)', 'units':'=0 wet, =1 dry', "standard_name": ""}),

        "CBH": (('n_crops',), CBH, {'long_name': 'Cloud base height', 'units':'m', "standard_name": "CBH"}),
        'CBH_std':(('n_crops',), CBH_std, {'long_name': 'Cloud base height standard deviation', 'units':'m', "standard_name": "CBH_standard_dev"}),
        'q':(('n_crops','levels'), q_profile, {'long_name': 'Specific humidity', 'units':'kg kg**-1',}),
        'q_std':(('n_crops','levels'), q_profile_std, {'long_name': 'Specific humidity standard deviation', 'units':'kg kg**-1',}),
        't_std':(('n_crops','levels'), t_profile_std, {'long_name': 'Temperature standard deviation', 'units':'K'}),
        't':(('n_crops','levels'), t_profile, {'long_name': 'Temperature', 'standard_name':'Temperature', 'units':'K'}),
        'w_std':(('n_crops','levels'), w_profile_std, {'long_name': 'vertical velocity standard deviation', 'units':'ms-1'}),
        'w':(('n_crops','levels'), w_profile, {'long_name': 'Vertical velocity', 'standard_name':'w', 'units':'ms-1'}),
        'u_std':(('n_crops','levels'), u_profile_std, {'long_name': 'U component of wind standard deviation', 'units':'ms-1'}),
        'u':(('n_crops','levels'), u_profile, {'long_name': 'U component of wind', 'standard_name':'eastward wind', 'units':'ms-1'}),
        'v_std':(('n_crops','levels'), v_profile_std, {'long_name': 'v component of wind standard deviation', 'units':'ms-1'}),
        'v':(('n_crops','levels'), v_profile, {'long_name': 'V component of wind', 'standard_name':'northward_wind', 'units':'ms-1'}),
        'rh_std':(('n_crops','levels'), rh_profile_std, {'long_name': 'rel hum standard deviation', 'units':'%'}),
        'rh':(('n_crops','levels'), rh_profile, {'long_name': 'relative humidity', 'standard_name':'rel hum', 'units':'%'}),  
        'clwc_std':(('n_crops','levels'), clwc_profile_std, {'long_name': 'specific cloud liquid water content standard deviation', 'units':'kg kg**-1'}),
        'clwc':(('n_crops','levels'), clwc_profile, {'long_name': 'specific cloud liquid water content', 'standard_name':'rel hum', 'units':'kg kg**-1'}),
        'cc_std':(('n_crops','levels'), cc_profile_std, {'long_name': 'fraction of cloud cover standard deviation', 'units':'kg kg**-1'}),
        'cc':(('n_crops','levels'), cc_profile, {'long_name': 'raction of cloud cover', 'standard_name':'rel hum', 'units':'kg kg**-1'}),    
    },
    coords={
        "n_crops": (('n_crops',), np.arange(n_images) ,), # leave units intentionally blank, to be defined in the encoding
        "levels": (('levels',), height, {"axis": "pressure_level","positive": "up","units": "millibars", "long_name":'pressure_level'}),
    },
    attrs={'CREATED_BY'     : 'Claudia Acquistapace',
                    'CREATED_ON'       : str(datetime.now()),
                    'FILL_VALUE'       : 'NaN',
                    'PI_NAME'          : 'Claudia Acquistapace',
                    'PI_AFFILIATION'   : 'University of Cologne (UNI), Germany',
                    'PI_ADDRESS'       : 'Institute for geophysics and meteorology, Pohligstrasse 3, 50969 Koeln',
                    'PI_MAIL'          : 'cacquist@meteo.uni-koeln.de',
                    'DATA_DESCRIPTION' : 'ERA5 variables for all the crops of the selected satellite position ',
                    'DATA_DISCIPLINE'  : 'Atmospheric Physics - Remote Sensing Radar Profiler',
                    'DATA_GROUP'       : 'Model: reanalysis',
                    'DATA_LOCATION'    : 'Atlantic Ocean - Eurec4a campaign domain',
                    'DATA_SOURCE'      : 'ERA5',
                    'DATA_PROCESSING'  : 'https://github.com/ClauClouds/ML_work_DC',
                    'COMMENT'          : '' }
    )




    # assign additional attributes following CF convention
    crop_data = crop_data.assign_attrs({
        "Conventions": "CF-1.8",
        "title": crop_data.attrs["DATA_DESCRIPTION"],
        "institution": crop_data.attrs["PI_AFFILIATION"],
        "history": "".join([
            "source: " + crop_data.attrs["DATA_SOURCE"] + "\n",
            "processing: " + crop_data.attrs["DATA_PROCESSING"] + "\n",
            " adapted to enhance CF compatibility\n",
        ]),  # the idea of this attribute is that each applied transformation is appended to create something like a log
        "featureType": "satellite-era5",
    })

    # storing ncdf data
    crop_data.to_netcdf(path_out+area+'_era5.nc')



wet_only
0 _2 20173041300408
1 _3 20173041300408
2 _5 20173041300408
3 _4 20173041300408
4 _1 20173041300408
5 _2 20173041300408
6 _5 20173041300408
7 _1 20173041300408
8 _2 20173041300408
9 _3 20173041300408
10 _4 20173041300408
11 _5 20173041300408
12 _1 20173041300408
13 _2 20173041300408
14 _1 20173041300408
15 _2 20173041300408
16 _4 20173041300408
17 _5 20173041300408
18 _1 20173041300408
19 _2 20173041300408
20 _3 20173041300408
21 _4 20173041300408
22 _5 20173041300408
23 _1 20173041300408
24 _2 20173041300408
25 _4 20173041300408
26 _1 20173041300408
27 _2 20173041300408
28 _4 20173041300408
29 _1 20173041300408
30 _2 20173041300408
31 _1 20173041300408
32 _5 20173041300408
33 _h 20173041300408
34 _5 20173041300408
35 _h 20173041300408
36 _2 20173041300408
37 _3 20173041300408
38 _4 20173041300408
39 _h 20173041300408
40 _4 20173041300408
41 _5 20173041300408
42 _h 20173041300408
43 _1 20173041300408
44 _4 20173041300408
45 _5 20173041300408
46 _h 20173041300408
47 _5 20173041

In [ ]:
n_features_same_coords = len(file_lat_list)
            print(ind_images, n_features_same_coords)
            # check if the image id selected has lats and lons
            if n_features_same_coords > 0:
                
                # loop on the positions on the map of the various images:
                for ind_map in range(n_features_same_coords):
                    # storing position 
                    pos_arr.append(ind_map)
                    
                    # reading lats/lons for the id_image
                    lat_data = np.load(file_lat_list[ind_map])
                    lon_data = np.load(file_lon_list[ind_map])
                    lat_max = np.nanmax(lat_data)
                    lat_min = np.nanmin(lat_data)
                    lon_min = np.nanmin(lon_data)
                    lon_max = np.nanmax(lon_data)

                    # reading era
                    profile_era5 = xr.open_dataset(era5_day_path+'profb_presslev_'+yy+mm+dd+'T'+hh+'00.nc')
                    surface_era5 = xr.open_dataset(era5_day_path+'surfskinvarb_'+yy+mm+dd+'T'+hh+'00.nc')

                    # selecting the area corresponding to the crop
                    surface_crop = surface_era5.where((surface_era5.latitude > lat_min)*(surface_era5.latitude <= lat_max) \
                                            * (surface_era5.longitude > lon_min) *(surface_era5.longitude <= lon_max))

                    profiles_crop = profile_era5.where((profile_era5.latitude > lat_min)*(profile_era5.latitude <= lat_max) \
                                            * (profile_era5.longitude > lon_min)*(profile_era5.longitude <= lon_max))

                    # making a plot of the era5 domain and the selected box
                    #fig, axs = plt.subplots(figsize=(10,10), constrained_layout=True)
                    #surface_era5.tcwv.plot(x='longitude', y='latitude')
                    #rect=mpatches.Rectangle((lon_max, lat_min),lon_min-lon_max,lat_max-lat_min,
                    #                fill = False,
                #                    color = "purple",
                #                    linewidth = 2)
                #    plt.gca().add_patch(rect)
                #    fig.savefig(path_out+id_image+'_'+type_im+'.png')

                    # calculating mean profiles and mean surface variables in the selected domain
                    SST.append(surface_crop.sst.mean(skipna='True'))
                    #SST_std.append(surface_crop.sst.std(skipna='True'))

                    TCWV.append(surface_crop.tcwv.mean(skipna='True'))
                    #TCWV_std.append(surface_crop.tcwv.std(skipna='True'))

                    TCC.append(surface_crop.tcc.mean(skipna='True'))
                    #TCC_std.append(surface_crop.tcc.std(skipna='True')

                    TCLW.append(surface_crop.tclw.mean(skipna='True'))
                    #TCLW_std.append(surface_crop.tclw.std(skipna='True')

                    TCRW.append(surface_crop.tcrw.mean(skipna='True'))
                    #TCRW_std.append(surface_crop.tcrw.std(skipna='True')

                    CBH.append(surface_crop.cbh.mean(skipna='True'))
                    #CBH_std.append(surface_crop.cbh.std(skipna='True')

                    q_profile.append(profiles_crop.q.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #q_profile_std.append(profiles_crop.q.std(dim=('longitude', 'latitude'), skipna='True')
                    t_profile.append(profiles_crop.t.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #t_profile_std.append(profiles_crop.t.std(dim=('longitude', 'latitude'), skipna='True')
                    w_profile.append(profiles_crop.w.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #w_profile_std.append(profiles_crop.w.std(dim=('longitude', 'latitude'), skipna='True')
                    u_profile.append(profiles_crop.u.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #u_profile_std.append(profiles_crop.u.std(dim=('longitude', 'latitude'), skipna='True')
                    v_profile.append(profiles_crop.v.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #v_profile_std.append(profiles_crop.v.std(dim=('longitude', 'latitude'), skipna='True')
                    rh_profile.append(profiles_crop.r.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #rh_profile_std.append(profiles_crop.r.std(dim=('longitude', 'latitude'), skipna='True')
                    clwc_profile.append(profiles_crop.clwc.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #clwc_profile_std.append(profiles_crop.clwc.std(dim=('longitude', 'latitude'), skipna='True')
                    cc_profile.append(profiles_crop.cc.mean(dim=('longitude', 'latitude'), skipna='True'))
                    #cc_profile_std.append(profiles_crop.cc.std(dim=('longitude', 'latitude'), skipna='True')
            
                                          
    
    # formatting datasets collected into arrays
    n_images = len(q_profile)
    q_profile_m = np.zeros((n_images, len(height)))
    t_profile_m = np.zeros((n_images, len(height)))
    w_profile_m = np.zeros((n_images, len(height)))
    rh_profile_m = np.zeros((n_images, len(height)))
    clwc_profile_m = np.zeros((n_images, len(height)))
    cc_profile_m = np.zeros((n_images, len(height)))
    u_profile_m = np.zeros((n_images, len(height)))
    v_profile_m = np.zeros((n_images, len(height)))
    for ind_profile in range(n_images):
        q_profile_m[ind_profile, :] =  q_profile[ind_profile]
        t_profile_m[ind_profile, :] =  t_profile[ind_profile]
        w_profile_m[ind_profile, :] =  w_profile[ind_profile]
        rh_profile_m[ind_profile, :] =  rh_profile[ind_profile]
        clwc_profile_m[ind_profile, :] = clwc_profile[ind_profile]
        cc_profile_m[ind_profile, :] =  cc_profile[ind_profile]
        u_profile_m[ind_profile, :] =  u_profile[ind_profile]
        v_profile_m[ind_profile, :] =  v_profile[ind_profile]
                                     
    # saving a ncdf file for each group of crops
    crop_data = xr.Dataset(
    data_vars={
        'im_names': (('n_crops',), np.asarray(im_name_arr), {'long_name': 'Names of image', 'units':''}),
        'time_stamps': (('n_crops',), np.asarray(datetime_arr), {'long_name': 'Names of image', 'units':''}),
        'pos_images': (('n_crops',), np.asarray(pos_arr), {'long_name': 'Names of image', 'units':''}),
        "SST": (('n_crops',), np.asarray(SST), {'long_name': 'Sea surface temperature ', 'units':'K', "standard_name": "SST"}),
        #'SST_std':(('n_crops',), np.asarray(SST_std), {'long_name': 'Sea suface temperature standard deviation', 'units':'K', "standard_name": "SST_standard_dev"}),
        "TCWV": (('n_crops',), np.asarray(TCWV), {'long_name': 'Total column water vapor ', 'units':'kg m**-2', "standard_name": "TCWV"}),
        #'TCWV_std':(('n_crops',), np.asarray(TCWV_std), {'long_name': 'Total column water vapor standard deviation', 'units':'kg m**-2', "standard_name": "TCWV_standard_dev"}),
        "TCC": (('n_crops',), np.asarray(TCC), {'long_name': 'Total cloud cover', 'units':'', "standard_name": "TCC"}),
        #'TCC_std':(('n_crops',), np.asarray(TCC_std), {'long_name': 'Total cloud cover standard deviation', 'units':'', "standard_name": "TCC_standard_dev"}),
        "TCLW": (('n_crops',), np.asarray(TCLW), {'long_name': 'Total column cloud liquid water', 'units':'Kg m**-2', "standard_name": "TCLW"}),
        #'TCLW_std':(('n_crops',), np.asarray(TCLW_std), {'long_name': 'Total column cloud liquid water standard deviation', 'units':'Kg m**-2', "standard_name": "TCLW_standard_dev"}),
        "TCRW": (('n_crops',), np.asarray(TCRW), {'long_name': 'Total column rain water', 'units':'Kg m**-2', "standard_name": "TCRW"}),
        #'TCRW_std':(('n_crops',), np.asarray(TCRW_std), {'long_name': 'Total column rain water standard deviation', 'units':'Kg m**-2', "standard_name": "TCRW_standard_dev"}),
        'im_type':(('n_crops',), np.asarray(im_type), {'long_name': 'type of image (wet or dry)', 'units':'=0 wet, =1 dry', "standard_name": ""}),

        "CBH": (('n_crops',), CBH, {'long_name': 'Cloud base height', 'units':'m', "standard_name": "CBH"}),
        #'CBH_std':(('n_crops',), CBH_std, {'long_name': 'Cloud base height standard deviation', 'units':'m', "standard_name": "CBH_standard_dev"}),
        'q':(('n_crops','levels'), q_profile, {'long_name': 'Specific humidity', 'units':'kg kg**-1',}),
        #'q_std':(('n_crops','levels'), q_profile_std, {'long_name': 'Specific humidity standard deviation', 'units':'kg kg**-1',}),
        #'t_std':(('n_crops','levels'), t_profile_std, {'long_name': 'Temperature standard deviation', 'units':'K'}),
        't':(('n_crops','levels'), t_profile, {'long_name': 'Temperature', 'standard_name':'Temperature', 'units':'K'}),
        #'w_std':(('n_crops','levels'), w_profile_std, {'long_name': 'vertical velocity standard deviation', 'units':'ms-1'}),
        'w':(('n_crops','levels'), w_profile, {'long_name': 'Vertical velocity', 'standard_name':'w', 'units':'ms-1'}),
        #'u_std':(('n_crops','levels'), u_profile_std, {'long_name': 'U component of wind standard deviation', 'units':'ms-1'}),
        'u':(('n_crops','levels'), u_profile, {'long_name': 'U component of wind', 'standard_name':'eastward wind', 'units':'ms-1'}),
        #'v_std':(('n_crops','levels'), v_profile_std, {'long_name': 'v component of wind standard deviation', 'units':'ms-1'}),
        'v':(('n_crops','levels'), v_profile, {'long_name': 'V component of wind', 'standard_name':'northward_wind', 'units':'ms-1'}),
        #'rh_std':(('n_crops','levels'), rh_profile_std, {'long_name': 'rel hum standard deviation', 'units':'%'}),
        'rh':(('n_crops','levels'), rh_profile, {'long_name': 'relative humidity', 'standard_name':'rel hum', 'units':'%'}),  
        #'clwc_std':(('n_crops','levels'), clwc_profile_std, {'long_name': 'specific cloud liquid water content standard deviation', 'units':'kg kg**-1'}),
        'clwc':(('n_crops','levels'), clwc_profile, {'long_name': 'specific cloud liquid water content', 'standard_name':'rel hum', 'units':'kg kg**-1'}),
        #'cc_std':(('n_crops','levels'), cc_profile_std, {'long_name': 'fraction of cloud cover standard deviation', 'units':'kg kg**-1'}),
        'cc':(('n_crops','levels'), cc_profile, {'long_name': 'raction of cloud cover', 'standard_name':'rel hum', 'units':'kg kg**-1'}),    
    },
    coords={
        "n_crops": (('n_crops',), np.arange(n_images) ,), # leave units intentionally blank, to be defined in the encoding
        "levels": (('levels',), height, {"axis": "pressure_level","positive": "up","units": "millibars", "long_name":'pressure_level'}),
    },
    attrs={'CREATED_BY'     : 'Claudia Acquistapace',
                    'CREATED_ON'       : str(datetime.now()),
                    'FILL_VALUE'       : 'NaN',
                    'PI_NAME'          : 'Claudia Acquistapace',
                    'PI_AFFILIATION'   : 'University of Cologne (UNI), Germany',
                    'PI_ADDRESS'       : 'Institute for geophysics and meteorology, Pohligstrasse 3, 50969 Koeln',
                    'PI_MAIL'          : 'cacquist@meteo.uni-koeln.de',
                    'DATA_DESCRIPTION' : 'ERA5 variables for all the crops of the selected satellite position ',
                    'DATA_DISCIPLINE'  : 'Atmospheric Physics - Remote Sensing Radar Profiler',
                    'DATA_GROUP'       : 'Model: reanalysis',
                    'DATA_LOCATION'    : 'Atlantic Ocean - Eurec4a campaign domain',
                    'DATA_SOURCE'      : 'ERA5',
                    'DATA_PROCESSING'  : 'https://github.com/ClauClouds/ML_work_DC',
                    'COMMENT'          : '' }
    )

    # assign additional attributes following CF convention
    crop_data = crop_data.assign_attrs({
        "Conventions": "CF-1.8",
        "title": crop_data.attrs["DATA_DESCRIPTION"],
        "institution": crop_data.attrs["PI_AFFILIATION"],
        "history": "".join([
            "source: " + crop_data.attrs["DATA_SOURCE"] + "\n",
            "processing: " + crop_data.attrs["DATA_PROCESSING"] + "\n",
            " adapted to enhance CF compatibility\n",
        ]),  # the idea of this attribute is that each applied transformation is appended to create something like a log
        "featureType": "satellite-era5",
    })

    # storing ncdf data
    crop_data.to_netcdf(path_out+area+'_era5.nc')
    print('saved file: '+path_out+area+'_era5.nc')

In [ ]:
# inizialization of all output variables
    # initializing output variables
    SST = np.zeros(n_images)
    im_type = np.zeros(n_images)
    im_name_arr = np.zeros(n_images)
    TCWV =np.zeros(n_images)
    TCC = np.zeros(n_images)
    TCLW =np.zeros(n_images)
    TCRW = np.zeros(n_images)
    CBH = np.zeros(n_images)
    SST_std = np.zeros(n_images)
    TCWV_std = np.zeros(n_images)
    TCC_std = np.zeros(n_images)
    TCLW_std =np.zeros(n_images)
    TCRW_std = np.zeros(n_images)
    CBH_std = np.zeros(n_images)
    q_profile = np.zeros((n_images, len(height)))
    t_profile = np.zeros((n_images, len(height)))
    q_profile_std = np.zeros((n_images, len(height)))
    t_profile_std = np.zeros((n_images, len(height)))
    w_profile = np.zeros((n_images, len(height)))
    rh_profile = np.zeros((n_images, len(height)))
    w_profile_std = np.zeros((n_images, len(height)))
    rh_profile_std = np.zeros((n_images, len(height)))
    clwc_profile = np.zeros((n_images, len(height)))
    clwc_profile_std = np.zeros((n_images, len(height)))
    cc_profile = np.zeros((n_images, len(height)))
    cc_profile_std = np.zeros((n_images, len(height)))
    u_profile = np.zeros((n_images, len(height)))
    u_profile_std = np.zeros((n_images, len(height)))
    v_profile = np.zeros((n_images, len(height)))
    v_profile_std = np.zeros((n_images, len(height)))
    SST.fill(np.nan)
    TCWV.fill(np.nan)
    TCC.fill(np.nan)
    TCLW.fill(np.nan)
    TCRW.fill(np.nan)
    CBH.fill(np.nan)
    SST_std.fill(np.nan)
    TCWV_std.fill(np.nan)
    TCC_std.fill(np.nan)
    TCLW_std.fill(np.nan)
    TCRW_std.fill(np.nan)
    CBH_std.fill(np.nan)
    q_profile.fill(np.nan)
    t_profile.fill(np.nan)
    q_profile_std.fill(np.nan)
    t_profile_std.fill(np.nan)
    w_profile.fill(np.nan)
    rh_profile.fill(np.nan)
    w_profile_std.fill(np.nan)
    rh_profile_std.fill(np.nan)
    clwc_profile.fill(np.nan)
    clwc_profile_std.fill(np.nan)
    u_profile.fill(np.nan)
    u_profile_std.fill(np.nan)
    v_profile.fill(np.nan)
    v_profile_std.fill(np.nan)
    im_name_arr.fill(np.nan)